In [58]:
import pandas as pd 
import numpy as np

In [59]:
data=pd.read_csv(r"C:\Users\sak\Documents\current_hackathon\bank_ruptcy\Bankruptcy dataset\1year.csv")

###### above feature set i have gone through some feature engineering steps
###### missing imputation------>gaussian transformation ----->decision tree descritisation------>scaling --->feature selection
[['Attr5', 'Attr11', 'Attr21', 'Attr22', 'Attr24', 'Attr27', 'Attr33', 'Attr40', 'Attr46', 'Attr55']] i got these features

###### but important analysis i found that feature set using these above feature engineering steps giving best result on original data aftter imputation only

In [60]:
year_1=data[['Attr5', 'Attr11', 'Attr21', 'Attr22', 'Attr24', 'Attr27', 'Attr33', 'Attr40', 'Attr46', 'Attr55','class']]

In [61]:
k1=year_1.columns.tolist()

In [62]:
#replacing each ? to NAN
for i in k1:
    year_1[i] = year_1[i].replace('?' , np.nan)

C:\Users\sak\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [63]:
#converting object data into numericals(float64)
year_1=year_1.apply(lambda col:pd.to_numeric(col,errors='coerce'))

In [64]:
year_1

,Attr5,Attr11,Attr21,Attr22,Attr24,Attr27,Attr33,Attr40,Attr46,Attr55,class
0,32.3510,0.249760,1.2479,0.214020,0.477060,1.45820,3.8772,0.662950,1.522500,348690.00,0
1,14.7860,0.261140,1.4293,0.248060,NaN,88.44400,2.9876,0.086422,1.125200,2304.60,0
2,-1.1523,0.312580,1.4283,0.302600,NaN,86.01100,2.0630,0.322020,1.010100,6332.70,0
3,51.9520,0.092704,1.5069,0.115500,0.171930,0.94076,3.9948,0.401390,1.569600,20545.00,0
4,-7.3128,0.187320,NaN,0.198320,0.187320,1.41380,2.4823,0.293040,0.957870,3186.60,0
...,...,...,...,...,...,...,...,...,...,...,...
7007,221.0100,0.045892,1.0877,0.046483,0.045892,NaN,6.3098,0.009419,4.636700,5775.70,1
7008,-44.4670,0.033243,NaN,-0.143840,0.001091,-4.47370,3.0876,0.013751,0.386740,14.00,1
7009,-7.2952,-0.067633,1.0377,-0.076312,-0.038146,-3.35570,14.5880,0.036787,0.234050,-396.00,1
7010,-88.3820,0.215510,NaN,0.189190,0.138090,2.44380,2.9623,0.041756,0.054665,-217.42,1


#####  so here i m building model on entire dataset for , so i m not divinding into train and test dataset , so that i do have ample amount of data to train our model more accurately  

In [65]:
predictors=['Attr5', 'Attr11', 'Attr21', 'Attr22', 'Attr24', 'Attr27', 'Attr33', 'Attr40', 'Attr46', 'Attr55']
target=['class']
X=year_1[predictors]
y=year_1[target]

In [66]:
#X
#y

In [67]:
from sklearn.impute import SimpleImputer

In [68]:
# Now we impute the missing values with SimpleImputer

# create an instance of the simple imputer
# we indicate that we want to impute with the median
mean_imputer = SimpleImputer(strategy='mean',)

# we fit and transform the predictors dataframe
# the imputer will learn the mean of all variables
X_new=mean_imputer.fit_transform(X[predictors])

In [69]:
y_1=y.copy()
X_1=pd.DataFrame(X_new, columns=predictors)

##### over sampling through SMOTE is performed 

In [70]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2) 
X_res, y_res = sm.fit_sample(X_1, y_1)

In [71]:
len(X_res)

13512

In [72]:
len(y_res)

13512

###### Note:- since , i have not splited my data into train-test dataset so my independent and dependent data are in pandas dataframe so in order to implement in XGBoostingClassifier algo in pickled , model will not take dataframe data as input to predict , so i need to convert X(input data ) into numpy array , i m not converting y into numpy array¶

In [73]:
#type(X_res)
type(y_res)

pandas.core.frame.DataFrame

In [74]:
X=np.array(X_res)
y=np.array(y_res)

In [75]:
#type(X)
#type(y)

In [76]:
import seaborn as sns
import xgboost as xgb
# Classification metrices
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report, precision_score,recall_score,f1_score

In [77]:
xgb.__version__

'1.1.1'

#### functionalizing the model 

In [78]:
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
import pickle

In [79]:
# Functionalize model fittting

def FitModel(X,Y,algo_name,algorithm,gridSearchParams,cv):
    np.random.seed(10)
    x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size = 0.2)
    
    
    grid = GridSearchCV(
        estimator=algorithm,
        param_grid=gridSearchParams,
        cv=cv, scoring='accuracy', verbose=1, n_jobs=-1)
    
    
    grid_result = grid.fit(x_train, y_train)
    best_params = grid_result.best_params_
    pred = grid_result.predict(x_test)
    cm = confusion_matrix(y_test, pred)
   # metrics =grid_result.gr
    print(pred)
    pickle.dump(grid_result,open(algo_name,'wb'))
   
    print('Best Params :',best_params)
    print('Classification Report :',classification_report(y_test,pred))
    print('Accuracy Score : ' + str(accuracy_score(y_test,pred)))
    print('Confusion Matrix : \n', cm)

In [80]:
param ={
            'n_estimators': [50, 100, 500, 1000, 2000],
           
        }
FitModel(X,y,'finalized_model5.pkl',XGBClassifier(),param,cv=5)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  2.2min finished
C:\Users\sak\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[1 1 0 ... 1 1 0]
Best Params : {'n_estimators': 500}
Classification Report :               precision    recall  f1-score   support

           0       0.99      0.97      0.98      1361
           1       0.97      0.99      0.98      1342

    accuracy                           0.98      2703
   macro avg       0.98      0.98      0.98      2703
weighted avg       0.98      0.98      0.98      2703

Accuracy Score : 0.9800221975582686
Confusion Matrix : 
 [[1326   35]
 [  19 1323]]


### Reloading the Saved Model

In [81]:

loaded_model = pickle.load(open("finalized_model5.pkl","rb"))

In [85]:
pred = loaded_model.predict(np.array([[32.3510,0.249760,1.2479,0.214020,0.477060,1.45820,3.8772,0.662950,1.522500,348690.00]]))

In [86]:
print(pred)

[0]


In [87]:
def change (prediction):
    if prediction==1:
        return 'yes'
    else:
        return 'No'

In [91]:
print('some of the features of company:-')
print('[(cash + short-term securities + receivables - short-term liabilities) / (operating expenses - depreciation)] * 365 = 32.3510')
print('(gross profit + extraordinary items + financial expenses) / total assets = 0.249760')
print('sales (n) / sales (n-1) = 1.2479')
print('profit on operating activities / total assets = 0.214020')
print('gross profit (in 3 years) / total assets = 0.477060')
print('profit on operating activities / financial expenses = 1.45820')
print('operating expenses / short-term liabilities = 3.8772')
print('(current assets - inventory - receivables) / short-term liabilities = 0.662950')
print('(current assets - inventory) / short-term liabilities = 1.522500')
print(' (sales - cost of products sold) / sales = 348690.00')
print('\n\n')

print('will company go for bankruptcy ? {}'.format(change(pred)))

some of the features of company:-
[(cash + short-term securities + receivables - short-term liabilities) / (operating expenses - depreciation)] * 365 = 32.3510
(gross profit + extraordinary items + financial expenses) / total assets = 0.249760
sales (n) / sales (n-1) = 1.2479
profit on operating activities / total assets = 0.214020
gross profit (in 3 years) / total assets = 0.477060
profit on operating activities / financial expenses = 1.45820
operating expenses / short-term liabilities = 3.8772
(current assets - inventory - receivables) / short-term liabilities = 0.662950
(current assets - inventory) / short-term liabilities = 1.522500
 (sales - cost of products sold) / sales = 348690.00



will company go for bankruptcy ? No
